In [1]:
Company.objects.all()

<QuerySet [<Company: 161ab539-eea0-4c7c-8e9f-99058261f850: CCR - SPVias>, <Company: 385c3404-2c8b-4987-8015-9b93e044aa6d: CCR - AutoBAn>, <Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>, <Company: be176150-b4da-4162-9eec-67f6db3f5612: CCR - RodoAnel>]>

Deve-se executar as seis primeiras células para carregar funções utilizadas para rodar o script

In [2]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from simple_history.utils import bulk_create_with_history
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from helpers.histories import bulk_update_with_history
import pytz
from helpers.signals import auto_add_number

In [3]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [4]:
from django.contrib.gis.geos import Point, LineString as DjangoLineString
from django.contrib.gis.db.models.functions import Distance
# from turfpy.misc import nearest_point_on_line
# from turfpy.measurement import length
from geojson import LineString, loads
from helpers.histories import bulk_update_with_history
from tqdm.notebook import tqdm

In [5]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [6]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [7]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [8]:
def findIndex(flist, func):
    for i,v in enumerate(flist):
        if func(v): 
            return i
    return -1

In [9]:
def point_to_km(lng, lat, road):
    point = Point(lng, lat, srid=4326)
    if isinstance(road,str):
        roads = Road.objects.filter(company=company,name__icontains=road).annotate(distance=Distance(point, 'path')).order_by('distance')
        road = roads[0]
    else:
        road = roads[0]
    path = loads(road.path.geojson)
    point_geojson = loads(point.geojson)
    road_marks = list(road.marks.values())
    snap_point = nearest_point_on_line(path, point_geojson)
    

    road_markers = sorted([a for a in road_marks if 'index' in a or ('index' in a and a['index'] == 0)], key=lambda x: int(x['key']))
#     road_markers = sorted([a for a in road_marks if a['index'] or a['index'] == 0], key=lambda x: int(x['key']))
    rm_index = findIndex(road_markers, lambda x: x['index'] > snap_point['properties']['index'])
    
    if (rm_index == -1):
        rm_index = len(road_markers) - 1;
    
    km_marker_start = road_markers[rm_index - 1]
    km_marker_end = road_markers[rm_index]
    
    if km_marker_start['index'] == road_markers[-1]['index']: 
        snap = road_markers[-1]
        km = snap['km']
    elif km_marker_end['index'] == road_markers[0]['index']:
        snap = road_markers[0]
        km = snap['km']
    else:
        test = path['coordinates'][km_marker_start['index']:km_marker_end['index'] + 1]
        marker_path = LineString(test)
        snap = nearest_point_on_line(marker_path, point_geojson)        
        ls_length = length(marker_path, units="km")
        km = ((snap['properties']['location'] / ls_length)        
            * (km_marker_end['km'] - km_marker_start['km'])
            + km_marker_start['km'])
    
    return road, road.name, point, round(km, 3)

In [10]:
from collections import defaultdict
from django.db.models.signals import *


class DisableSignals(object):
    def __init__(self, disabled_signals=None):
        self.stashed_signals = defaultdict(list)
        self.disabled_signals = disabled_signals or [
            pre_init, post_init,
            pre_save, post_save,
            pre_delete, post_delete,
            pre_migrate, post_migrate,
        ]

    def __enter__(self):
        for signal in self.disabled_signals:
            self.disconnect(signal)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for signal in list(self.stashed_signals):
            self.reconnect(signal)

    def disconnect(self, signal):
        self.stashed_signals[signal] = signal.receivers
        signal.receivers = []

    def reconnect(self, signal):
        signal.receivers = self.stashed_signals.get(signal, [])
        del self.stashed_signals[signal]

In [11]:
def get_connected_reference(company, resource, key, value):
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [12]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [13]:
def get_sign_color(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.capitalize().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [14]:
def get_sign_film(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    value=value.upper().strip()
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

 Verificando se está no ambiente de produção

In [15]:
!cat .env

STAGE=CCR_PRODUCTION


Escolhendo nome do arquivo Excel para carregar o Inventário

In [75]:
filename='OAC BR 116_Template_rev01'
# road_name='470'

Escolhendo a aba da planilha do Excel

In [76]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[1]
ws = wb[sheetname]

Testando planilha carregada do Excel

In [77]:
print(ws['A1'].value)

Identificação ANTT


Carregando os valores no script

In [78]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        
    values.append(obj)

Definindo valores da importação

In [79]:
company = Company.objects.get(name='CCR - Rio Sp')

#Classe Drenagem
occurrence_type_ficha = OccurrenceType.objects.get(name='Monitoração de Drenagem Profunda Ficha Poder Concedente',company=company)
occurrence_type_elemento = OccurrenceType.objects.get(name='Monitoração Drenagem Profunda Elemento',company=company)

user=User.objects.get(username='rlcs')
status=ServiceOrderActionStatus.objects.get(companies=company, name='Identificado')
firm=Firm.objects.get(name='Civil: SGOAC',company=company)
step=ApprovalStep.objects.get(approval_flow__company=company, name='Criado')
company,step

(<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>,
 <ApprovalStep: [CCR - Rio Sp]: Criado>)

In [80]:
# Reporting.objects.filter(company=company,occurrence_type=occurrence_type_elemento,
#                          form_data__code='CCS 116 RJ 262+621 N 1',
#                          created_at__gte='2023-12-13').count()

In [81]:
len(values),values[0]

(7605,
 {'Identificação ANTT': 'BU 116 RJ 215+140 N 1',
  'Código do Inventário para vinculo com apontamento': 'CCR-Inv-2023.161388',
  'km': '215+140',
  'km final': '215+120',
  'Status': 'Identificado',
  'Equipe': 'Civil: SGOAC',
  'Encontrado em': datetime.datetime(2024, 1, 5, 0, 0),
  'Executado em': datetime.datetime(2024, 1, 5, 0, 0),
  'Sentido': 'NORTE',
  'Classe': 'Monitoração de Drenagem Profunda',
  'Faixa': 'Não se aplica',
  'Rodovia': 'BR-116 RJ',
  'Elemento': 'OAC',
  'Extensão': 71.89,
  'Formato': 'TUBULAR',
  'Tipo de Bueiro (Celular)': None,
  'Tipo de Bueiro (Tubular)': 'BSTC',
  'Marginal': 'Não',
  'Latitude Montante': -22.7100723,
  'Longitude Montante': -43.7363794,
  'X UTM': None,
  'Y UTM': None,
  'Latitude Jusante': -22.7101837,
  'Longitude Jusante': -43.735782,
  'X UTM Jusante': None,
  'Y UTM Jusante': None,
  'Zona': None,
  'Estrutura Entrada': 'CCT',
  'Código Montante': 'BU 116 RJ 215+140 N 1',
  'Ambiente Montante': None,
  'Altura montante (m)

Criando itens de Inventário

In [82]:
var = 'BSCC'
[get_value('Tipo de Bueiro (Celular)', occurrence_type_ficha, v.strip()) for v in var.split(',')]

['1']

In [83]:
objects=[]
erros=[]
r=[]
r_mon=[]
r_jus=[]
rir_mon=[]
rir_jus=[]
for index, a in enumerate(tqdm(values)):
#     print(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']))
    km=float(a['km'].replace('+','.'))
    end_km=float(a['km final'].replace('+','.')) if a['km final'] is not None else None
    point, road = get_road_coordinates(a['Rodovia'], km, str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])), company)
    
    try:
        
        rep=Reporting(
                    company=company,
                    occurrence_type=occurrence_type_ficha,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                    road=road,
                    parent=Reporting.objects.filter(number=a['Código do Inventário para vinculo com apontamento'].strip(),
                                                    company=company).first() if a.get('Código do Inventário para vinculo com apontamento') else None,
                    road_name=a['Rodovia'],
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
                    approval_step=step,
                    found_at=a['Encontrado em'],
                    executed_at=a['Executado em'],
                    km=float(km),
                    end_km_manually_specified=True if a['km final'] is not None else False,
                    end_km=end_km if a['km final'] is not None else None,
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        "id_ccr_antt": a["Identificação ANTT"],
                        "element": a["Elemento"],
                        "length": a["Extensão"],
                        "format": [get_value('Formato', occurrence_type_ficha, v.strip().capitalize()) for v in a["Formato"].split(',')] if a.get("Formato") is not None else None,
#                         "format": get_value('Formato', occurrence_type_ficha, a["Formato"].strip().capitalize()) if a.get("Formato") is not None else None,
                        "other_format": a.get("Outro Formato"),
                        "hole_kind_celular":[get_value('Tipo de Bueiro (Celular)', occurrence_type_ficha, v.strip()) for v in a["Tipo de Bueiro (Celular)"].split(',')] if a.get("Tipo de Bueiro (Celular)") is not None else None,
                        "hole_kind_tubular":[get_value('Tipo de Bueiro (Tubular)', occurrence_type_ficha, v.strip()) for v in a["Tipo de Bueiro (Tubular)"].split(',')] if a.get("Tipo de Bueiro (Tubular)") is not None else None,
                        "hole_kind_ovoide":[get_value('Tipo de Bueiro (Ovóide)', occurrence_type_ficha, v.strip()) for v in a["Tipo de Bueiro (Ovóide)"].split(',')] if a.get("Tipo de Bueiro (Ovóide)") is not None else None,
                        "hole_kind_other":[get_value('Tipo de Bueiro', occurrence_type_ficha, v.strip()) for v in a["Tipo de Bueiro"].split(',')] if a.get("Tipo de Bueiro") is not None else None,
#                         "hole_kind_celular": get_value('Tipo de Bueiro (Celular)', occurrence_type_ficha, a["Tipo de Bueiro (Celular)"].strip()) if a.get("Tipo de Bueiro (Celular)") is not None else None,
#                         "hole_kind_tubular": get_value('Tipo de Bueiro (Tubular)', occurrence_type_ficha, a["Tipo de Bueiro (Tubular)"].strip()) if a.get("Tipo de Bueiro (Tubular)") is not None else None,
#                         "hole_kind_ovoide": get_value('Tipo de Bueiro (Ovóide)', occurrence_type_ficha, a["Tipo de Bueiro (Ovóide)"].strip()) if a.get("Tipo de Bueiro (Ovóide)") is not None else None,
#                         "hole_kind_other": get_value('Tipo de Bueiro', occurrence_type_ficha, a["Tipo de Bueiro"].strip()) if a.get("Tipo de Bueiro") is not None else None,
                        "passarel": get_value('Travessia', occurrence_type_ficha, a["Travessia"].strip()) if a.get("Travessia") is not None else None,
                        "marginal": get_value('Marginal', occurrence_type_ficha, a["Marginal"].strip()) if a.get("Marginal") is not None else None,
                        "sheet_classification": get_value('Estado Geral de Conservação', occurrence_type_ficha, a["Estado Geral de Conservação"].strip().capitalize()) if a.get("Estado Geral de Conservação") is not None else None,
                        "notes": a["Observações"]                                  
                    })
#         rep.save()
        auto_add_number(rep, "RP_name_format")
        r.append(rep)
        
        # Verificação Montante, caso não exista cria
        
        rep_montante=Reporting.objects.filter(occurrence_type=occurrence_type_elemento,form_data__code=a.get('Código Montante'))
        
        if not rep_montante.exists() and a.get('Código Montante'):
            if a.get('Latitude Montante') and a.get('Longitude Montante'):
                point=Point(float(a.get('Longitude Montante')), float(a.get('Latitude Montante')), srid=4326)
#                 _,_,point,_ =point_to_km(float(a.get('Longitude Montante')),float(a.get('Latitude Montante')),a['Rodovia'])
            rep_montante=Reporting(
                        company=company,
                        occurrence_type=occurrence_type_elemento,
                        lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                        road=road,
                        parent=Reporting.objects.filter(form_data__code=a.get('Código Montante'),
                                 company=company,
                                 occurrence_type__occurrence_kind='2').first(),
                        road_name=a['Rodovia'],
                        direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                        created_by=user,
                        found_at=a['Encontrado em'],
                        executed_at=a['Executado em'],
                        km=float(km),
                        end_km_manually_specified=True if a['km final'] is not None else False,
                        end_km=end_km if a['km final'] is not None else None,
                        point=point,
                        firm=firm,
                        status=status,
                        approval_step=step,
                        form_data={
                            "element": a["Elemento"],
                            "hole_classification": get_value('Estado Geral de Conservação da Caixa', occurrence_type_elemento, a["Estado de Conservação Montante"].strip().capitalize()) if a.get("Estado de Conservação Montante") is not None else None,
#                             "exclude_element": a["Elemento Excluído"]select
#                             "exclusion_motive": a["Motivo Exclusão"]textArea
                            "latitude": float(a.get('Latitude Montante',0)) if a.get('Latitude Montante') is not None else 0,
                            "longitude": float(a.get('Longitude Montante',0)) if a.get('Longitude Montante') is not None else 0,
                            "zone": get_value('Zona', occurrence_type_elemento, a["Zona"].strip()) if a.get('Zona') else None,
                            "x_utm": a["X UTM"],
                            "y_utm": a["Y UTM"],
                            "entry_struc": get_value('Estrutura Entrada', occurrence_type_elemento, a["Estrutura Entrada"]) if a.get('Estrutura Entrada') else None,
                            "ambient_mon": get_value('Ambiente', occurrence_type_elemento, a["Ambiente Montante"]) if a.get('Ambiente Montante') else None,
                            "height": a["Altura montante (m)"],
                            "width": a["Largura montante (m)"],
                            "diameter": a["Diâmetro montante (m)"],
                            "code": a["Código Montante"],
                            "material_rev_mont": get_value('Material Revestimento', occurrence_type_elemento, a["Material Revestimento Montante"]) if a.get('Material Revestimento Montante') else None,
                            "description_mat": a.get("Descrição do Material"),
                            "side_mont": get_value('Lado', occurrence_type_elemento, a["Lado Montante"]) if a.get('Lado Montante') else None,
                            "is_domain_stripmon": True if a.get('Está na faixa de domínio? (Montante)') and a.get('Está na faixa de domínio? (Montante)').lower()=='sim' else False,
                            "works_drownmon": True if a.get('Trabalha afogado? (Montante)') and a.get('Trabalha afogado? (Montante)').lower()=='sim' else False,
                            "cleaningmon": True if a.get('Limpeza - Montante') and a.get('Limpeza - Montante').lower()=='sim' else False,
                            "rocada_mon": True if a.get('Roçada - Montante ') and a.get('Roçada - Montante ').lower()=='sim' else False,
                            "desassoreamentomon": True if a.get('Desassoreamento - Montante') and a.get('Desassoreamento - Montante').lower()=='sim' else False,
                            "desobstrucaomon": True if a.get('Desobstrução - Montante') and a.get('Desobstrução - Montante').lower()=='sim' else False,
                            "forehead_damagemon": True if a.get('Testa ou ala danificada - Montante') and a.get('Testa ou ala danificada - Montante').lower()=='sim' else False,
                            "tube_damagemon": True if a.get('Tubulação danificada - Montante') and a.get('Tubulação danificada - Montante').lower()=='sim' else False,
                            "box_damagemon": True if a.get('Caixa danificada - Montante') and a.get('Caixa danificada - Montante').lower()=='sim' else False,
                            "erosionmon": True if a.get('Erosão - Montante') and a.get('Erosão - Montante').lower()=='sim' else False,
                            "brokenmon": True if a.get('Fissuras/trincas - Montante') and a.get('Fissuras/trincas - Montante').lower()=='sim' else False,
                            "cover_damagemon": True if a.get('Tampa Danificada/Inexist - Montante') and a.get('Tampa Danificada/Inexist - Montante').lower()=='sim' else False,                                            
                        })
            
#             rep_montante.save()
            auto_add_number(rep_montante, "RP_name_format")
            r_mon.append(rep_montante)
        else:
            rep_montante = rep_montante.first()
            
            
        # Verificação Jusante, caso não exista cria
        
        rep_jusante=Reporting.objects.filter(occurrence_type=occurrence_type_elemento,form_data__code=a.get('Código Jusante'))
        
        if not rep_jusante.exists() and a.get('Código Jusante'):
            if a.get('Latitude Jusante') and a.get('Longitude Jusante'):
                point=Point(float(a.get('Longitude Jusante')), float(a.get('Latitude Jusante')), srid=4326)
#                 _,_,point,_ =point_to_km(float(a.get('Longitude Jusante')),float(a.get('Latitude Jusante')),a['Rodovia'])
            rep_jusante=Reporting(
                        company=company,
                        occurrence_type=occurrence_type_elemento,
                        lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                        road=road,
                        road_name=a['Rodovia'],
                        parent=Reporting.objects.filter(form_data__code=a.get('Código Jusante'),
                                 company=company,
                                 occurrence_type__occurrence_kind='2').first(),
                        direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                        created_by=user,
                        found_at=a['Encontrado em'],
                        executed_at=a['Executado em'],
                        km=float(km),
                        end_km_manually_specified=True if a['km final'] is not None else False,
                        end_km=end_km if a['km final'] is not None else None,
                        point=point,
                        firm=firm,
                        status=status,
                        approval_step=step,
                        form_data={
                            "element": a["Elemento"],
                            "hole_classification": get_value('Estado Geral de Conservação da Caixa', occurrence_type_elemento, a["Estado de Conservação Jusante"].strip().capitalize()) if a.get("Estado de Conservação Jusante") is not None else None,
#                             "exclude_element": a["Elemento Excluído"]select
#                             "exclusion_motive": a["Motivo Exclusão"]textArea
                            "latitude": float(a.get('Latitude Jusante',0)) if a.get('Latitude Jusante') is not None else 0,
                            "longitude": float(a.get('Longitude Jusante',0)) if a.get('Longitude Jusante') is not None else 0,
                            "zone": get_value('Zona', occurrence_type_elemento, a["Zona"].strip()) if a.get('Zona') else None,
                            "x_utm": a["X UTM Jusante"],
                            "y_utm": a["Y UTM Jusante"],
                            "entry_struc": get_value('Estrutura Entrada', occurrence_type_elemento, a["Estrutura Entrada"]) if a.get('Estrutura Entrada') else None,
                            "ambient_mon": get_value('Ambiente', occurrence_type_elemento, a["Ambiente Jusante"]) if a.get('Ambiente Jusante') else None,
                            "height": a["Altura Jusante (m)"],
                            "width": a["Largura Jusante (m)"],
                            "diameter": a["Diâmetro Jusante (m)"],
                            "code": a["Código Jusante"],
                            "material_rev_mont": get_value('Material Revestimento', occurrence_type_elemento, a["Material Revestimento Jusante"]) if a.get('Material Revestimento Jusante') else None,
                            "description_mat": a.get("Descrição do Material"),
                            "side_mont": get_value('Lado', occurrence_type_elemento, a["Lado Jusante"]) if a.get('Lado Jusante') else None,
                            "is_domain_stripmon": True if a.get('Está na faixa de domínio? (Jusante)') and a.get('Está na faixa de domínio? (Jusante)').lower()=='sim' else False,
                            "works_drownmon": True if a.get('Trabalha afogado? (Jusante)') and a.get('Trabalha afogado? (Jusante)').lower()=='sim' else False,
                            "cleaningmon": True if a.get('Limpeza - Jusante') and a.get('Limpeza - Jusante').lower()=='sim' else False,
                            "rocada_mon": True if a.get('Roçada - Jusante ') and a.get('Roçada - Jusante ').lower()=='sim' else False,
                            "desassoreamentomon": True if a.get('Desassoreamento - Jusante') and a.get('Desassoreamento - Jusante').lower()=='sim' else False,
                            "desobstrucaomon": True if a.get('Desobstrução - Jusante') and a.get('Desobstrução - Jusante').lower()=='sim' else False,
                            "forehead_damagemon": True if a.get('Testa ou ala danificada - Jusante') and a.get('Testa ou ala danificada - Jusante').lower()=='sim' else False,
                            "tube_damagemon": True if a.get('Tubulação danificada - Jusante') and a.get('Tubulação danificada - Jusante').lower()=='sim' else False,
                            "box_damagemon": True if a.get('Caixa danificada - Jusante') and a.get('Caixa danificada - Jusante').lower()=='sim' else False,
                            "erosionmon": True if a.get('Erosão - Jusante') and a.get('Erosão - Jusante').lower()=='sim' else False,
                            "brokenmon": True if a.get('Fissuras/trincas - Jusante') and a.get('Fissuras/trincas - Jusante').lower()=='sim' else False,
                            "cover_damagemon": True if a.get('Tampa Danificada/Inexist - Jusante') and a.get('Tampa Danificada/Inexist - Jusante').lower()=='sim' else False,                                            
                        })
            
#             rep_jusante.save()
            auto_add_number(rep_jusante, "RP_name_format")
            r_jus.append(rep_jusante)
        else:
            rep_jusante = rep_jusante.first()
        
        
        # Relação ReportingInReporting e ReportingInRelation
        if a.get('Código Montante') and rep_montante:
            rep_relation = ReportingRelation.objects.get(name='Montante')
            rep_in_rep_montante=ReportingInReporting(parent=rep,child=rep_montante,reporting_relation=rep_relation)
#             rep_in_rep_montante.save()
            rir_mon.append(rep_in_rep_montante)
        if a.get('Código Jusante') and rep_jusante:
            rep_relation = ReportingRelation.objects.get(name='Jusante')
            rep_in_rep_jusante=ReportingInReporting(parent=rep,child=rep_jusante,reporting_relation=rep_relation)
#             rep_in_rep_jusante.save()
            rir_jus.append(rep_in_rep_jusante)
            
    except Exception as e:
        print(f'Linha: {index+2} - {e}')
#         a['number_ficha'] = rep.number if getattr(rep,'number') else ''
#         a['number_mon'] = rep_montante.number if getattr(rep_montante,'number') else ''
#         a['number_jus'] = rep_jusante.number if getattr(rep_jusante,'number') else ''
        erros.append(a)

  0%|          | 0/7605 [00:00<?, ?it/s]

AttributeError: 'NoneType' object has no attribute 'replace'

In [84]:
len(r)

7544

In [85]:
result1 = bulk_create_with_history(r, Reporting, default_user=user)
result2 = bulk_create_with_history(r_mon, Reporting, default_user=user)
result3 = bulk_create_with_history(r_jus, Reporting, default_user=user)
result4 = bulk_create_with_history(rir_mon, ReportingInReporting, default_user=user)
result5 = bulk_create_with_history(rir_jus, ReportingInReporting, default_user=user)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2024-01-05 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2024-01-05 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField Reporting.executed_at received a naive datetime (2024-01-06 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.c

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2024-01-05 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalReporting.found_at received a naive datetime (2024-01-05 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField HistoricalReporting.executed_at received a naive datetime (2024-01-06 00:00:00) while time zone support is active.
  warnings.warn("DateTimeField %s received a naiv

In [62]:
rep_edit=[]
for a in tqdm(result1):
    a.approval_step = step
    rep_edit.append(a)

  0%|          | 0/427 [00:00<?, ?it/s]

In [63]:
for a in tqdm(result2):
    a.approval_step = step
    rep_edit.append(a)

  0%|          | 0/427 [00:00<?, ?it/s]

In [64]:
for a in tqdm(result3):
    a.approval_step = step
    rep_edit.append(a)

  0%|          | 0/427 [00:00<?, ?it/s]

In [65]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=100,user=user)

In [ ]:
import pandas as pd
pd.DataFrame(data=erros).to_excel(filename+'_erros.xlsx')

# Verificação dos itens antes da importação

In [28]:
erros=[]
for index, a in enumerate(tqdm(values[5036:5040])):
#     print(float(a['Longitude_Inicial']),float(a['Latitude_Inicial']))
    km=float(a['km'].replace('+','.'))
    end_km=float(a['km final'].replace('+','.'))
    point, road = get_road_coordinates(a['Rodovia'], km, str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])), company)
    
    try:
        
        rep=Reporting(
                    company=company,
                    occurrence_type=occurrence_type_ficha,
                    lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                    road=road,
                    parent=Reporting.objects.filter(number=a['Código do Inventário para vinculo com apontamento'].strip(),
                                                    company=company).first() if a.get('Código do Inventário para vinculo com apontamento') else None,
                    road_name=a['Rodovia'],
                    direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                    created_by=user,
                    found_at=a['Encontrado em'],
                    executed_at=a['Executado em'],
                    km=float(km),
                    end_km_manually_specified=True if a['km final'] is not None else False,
                    end_km=float(end_km) if a['km final'] is not None else None,
                    point=point,
                    firm=firm,
                    status=status,
                    form_data={
                        "id_ccr_antt": a["Identificação ANTT"],
                        "element": a["Elemento"],
                        "length": a["Extensão"],
                        "format": get_value('Formato', occurrence_type_ficha, a["Formato"].strip().capitalize()) if a.get("Tipo de Bueiro (Celular)") is not None else None,
                        "other_format": a.get("Outro Formato"),
                        "hole_kind_celular": get_value('Tipo de Bueiro (Celular)', occurrence_type_ficha, a["Tipo de Bueiro (Celular)"].strip()) if a.get("Tipo de Bueiro (Celular)") is not None else None,
                        "hole_kind_tubular": get_value('Tipo de Bueiro (Tubular)', occurrence_type_ficha, a["Tipo de Bueiro (Tubular)"].strip()) if a.get("Tipo de Bueiro (Tubular)") is not None else None,
                        "hole_kind_ovoide": get_value('Tipo de Bueiro (Ovóide)', occurrence_type_ficha, a["Tipo de Bueiro (Ovóide)"].strip()) if a.get("Tipo de Bueiro (Ovóide)") is not None else None,
                        "hole_kind_other": get_value('Tipo de Bueiro', occurrence_type_ficha, a["Tipo de Bueiro"].strip()) if a.get("Tipo de Bueiro") is not None else None,
                        "passarel": get_value('Travessia', occurrence_type_ficha, a["Travessia"].strip()) if a.get("Travessia") is not None else None,
                        "marginal": get_value('Marginal', occurrence_type_ficha, a["Marginal"].strip()) if a.get("Marginal") is not None else None,
                        "sheet_classification": get_value('Estado Geral de Conservação', occurrence_type_ficha, a["Estado Geral de Conservação"].strip()) if a.get("Estado Geral de Conservação") is not None else None,
                        "notes": a["Observações"]                                  
                    })
#         rep.save()
        
        # Verificação Montante, caso não exista cria
        
        rep_montante=Reporting.objects.filter(occurrence_type=occurrence_type_elemento,form_data__code=a.get('Código Montante'))
        
        if not rep_montante.exists() and a.get('Código Montante'):
            if a.get('Latitude Montante') and a.get('Longitude Montante'):
                point=Point(float(a.get('Longitude Montante')), float(a.get('Latitude Montante')), srid=4326)
#                 _,_,point,_ =point_to_km(float(a.get('Longitude Montante')),float(a.get('Latitude Montante')),a['Rodovia'])
            rep_montante=Reporting(
                        company=company,
                        occurrence_type=occurrence_type_elemento,
                        lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                        road=road,
                        parent=Reporting.objects.filter(form_data__code=a.get('Código Montante'),
                                 company=company,
                                 occurrence_type__occurrence_kind='2').first(),
                        road_name=a['Rodovia'],
                        direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                        created_by=user,
                        found_at=a['Encontrado em'],
                        executed_at=a['Executado em'],
                        km=float(km),
                        end_km_manually_specified=True if a['km final'] is not None else False,
                        end_km=float(end_km) if a['km final'] is not None else None,
                        point=point,
                        firm=firm,
                        status=status,
                        form_data={
                            "element": a["Elemento"],
                            "hole_classification": get_value('Estado Geral de Conservação da Caixa', occurrence_type_elemento, a["Estado de Conservação Montante"].strip()) if a.get("Estado de Conservação Montante") is not None else None,
#                             "exclude_element": a["Elemento Excluído"]select
#                             "exclusion_motive": a["Motivo Exclusão"]textArea
                            "latitude": float(a.get('Latitude Montante',0)),
                            "longitude": float(a.get('Longitude Montante',0)),
                            "zone": get_value('Zona', occurrence_type_elemento, a["Zona"].strip()) if a.get('Zona') else None,
                            "x_utm": a["X UTM"],
                            "y_utm": a["Y UTM"],
                            "entry_struc": get_value('Estrutura Entrada', occurrence_type_elemento, a["Estrutura Entrada"]) if a.get('Estrutura Entrada') else None,
                            "ambient_mon": get_value('Ambiente', occurrence_type_elemento, a["Ambiente Montante"]) if a.get('Ambiente Montante') else None,
                            "height": a["Altura montante (m)"],
                            "width": a["Largura montante (m)"],
                            "diameter": a["Diâmetro montante (m)"],
                            "code": a["Código Montante"],
                            "material_rev_mont": get_value('Material Revestimento', occurrence_type_elemento, a["Material Revestimento Montante"]) if a.get('Material Revestimento Montante') else None,
                            "description_mat": a.get("Descrição do Material"),
                            "side_mont": get_value('Lado', occurrence_type_elemento, a["Lado Montante"]) if a.get('Lado Montante') else None,
                            "is_domain_stripmon": True if a.get('Está na faixa de domínio? (Montante)') and a.get('Está na faixa de domínio? (Montante)').lower()=='sim' else False,
                            "works_drownmon": True if a.get('Trabalha afogado? (Montante)') and a.get('Trabalha afogado? (Montante)').lower()=='sim' else False,
                            "cleaningmon": True if a.get('Limpeza - Montante') and a.get('Limpeza - Montante').lower()=='sim' else False,
                            "rocada_mon": True if a.get('Roçada - Montante ') and a.get('Roçada - Montante ').lower()=='sim' else False,
                            "desassoreamentomon": True if a.get('Desassoreamento - Montante') and a.get('Desassoreamento - Montante').lower()=='sim' else False,
                            "desobstrucaomon": True if a.get('Desobstrução - Montante') and a.get('Desobstrução - Montante').lower()=='sim' else False,
                            "forehead_damagemon": True if a.get('Testa ou ala danificada - Montante') and a.get('Testa ou ala danificada - Montante').lower()=='sim' else False,
                            "tube_damagemon": True if a.get('Tubulação danificada - Montante') and a.get('Tubulação danificada - Montante').lower()=='sim' else False,
                            "box_damagemon": True if a.get('Caixa danificada - Montante') and a.get('Caixa danificada - Montante').lower()=='sim' else False,
                            "erosionmon": True if a.get('Erosão - Montante') and a.get('Erosão - Montante').lower()=='sim' else False,
                            "brokenmon": True if a.get('Fissuras/trincas - Montante') and a.get('Fissuras/trincas - Montante').lower()=='sim' else False,
                            "cover_damagemon": True if a.get('Tampa Danificada/Inexist - Montante') and a.get('Tampa Danificada/Inexist - Montante').lower()=='sim' else False,                                            
                        })
            
#             rep_montante.save()
        else:
            rep_montante = rep_montante.first()
            
            
        # Verificação Jusante, caso não exista cria
        
        rep_jusante=Reporting.objects.filter(occurrence_type=occurrence_type_elemento,form_data__code=a.get('Código Jusante'))
        
        if not rep_jusante.exists() and a.get('Código Jusante'):
            if a.get('Latitude Jusante') and a.get('Longitude Jusante'):
                point=Point(float(a.get('Longitude Jusante')), float(a.get('Latitude Jusante')), srid=4326)
#                 _,_,point,_ =point_to_km(float(a.get('Longitude Jusante')),float(a.get('Latitude Jusante')),a['Rodovia'])
            rep_jusante=Reporting(
                        company=company,
                        occurrence_type=occurrence_type_elemento,
                        lane=str(get_connected_reference(company, 'reporting', 'lane', a['Faixa'])),
                        road=road,
                        road_name=a['Rodovia'],
                        parent=Reporting.objects.filter(form_data__code=a.get('Código Jusante'),
                                 company=company,
                                 occurrence_type__occurrence_kind='2').first(),
                        direction=str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'])),
                        created_by=user,
                        found_at=a['Encontrado em'],
                        executed_at=a['Executado em'],
                        km=float(km),
                        end_km_manually_specified=True if a['km final'] is not None else False,
                        end_km=float(end_km) if a['km final'] is not None else None,
                        point=point,
                        firm=firm,
                        status=status,
                        form_data={
                            "element": a["Elemento"],
                            "hole_classification": get_value('Estado Geral de Conservação da Caixa', occurrence_type_elemento, a["Estado de Conservação Jusante"].strip()) if a.get("Estado de Conservação Jusante") is not None else None,
#                             "exclude_element": a["Elemento Excluído"]select
#                             "exclusion_motive": a["Motivo Exclusão"]textArea
                            "latitude": float(a.get('Latitude Jusante',0)),
                            "longitude": float(a.get('Longitude Jusante',0)),
                            "zone": get_value('Zona', occurrence_type_elemento, a["Zona"].strip()) if a.get('Zona') else None,
                            "x_utm": a["X UTM Jusante"],
                            "y_utm": a["Y UTM Jusante"],
                            "entry_struc": get_value('Estrutura Entrada', occurrence_type_elemento, a["Estrutura Entrada"]) if a.get('Estrutura Entrada') else None,
                            "ambient_mon": get_value('Ambiente', occurrence_type_elemento, a["Ambiente Jusante"]) if a.get('Ambiente Jusante') else None,
                            "height": a["Altura Jusante (m)"],
                            "width": a["Largura Jusante (m)"],
                            "diameter": a["Diâmetro Jusante (m)"],
                            "code": a["Código Jusante"],
                            "material_rev_mont": get_value('Material Revestimento', occurrence_type_elemento, a["Material Revestimento Jusante"]) if a.get('Material Revestimento Jusante') else None,
                            "description_mat": a.get("Descrição do Material"),
                            "side_mont": get_value('Lado', occurrence_type_elemento, a["Lado Jusante"]) if a.get('Lado Jusante') else None,
                            "is_domain_stripmon": True if a.get('Está na faixa de domínio? (Jusante)') and a.get('Está na faixa de domínio? (Jusante)').lower()=='sim' else False,
                            "works_drownmon": True if a.get('Trabalha afogado? (Jusante)') and a.get('Trabalha afogado? (Jusante)').lower()=='sim' else False,
                            "cleaningmon": True if a.get('Limpeza - Jusante') and a.get('Limpeza - Jusante').lower()=='sim' else False,
                            "rocada_mon": True if a.get('Roçada - Jusante ') and a.get('Roçada - Jusante ').lower()=='sim' else False,
                            "desassoreamentomon": True if a.get('Desassoreamento - Jusante') and a.get('Desassoreamento - Jusante').lower()=='sim' else False,
                            "desobstrucaomon": True if a.get('Desobstrução - Jusante') and a.get('Desobstrução - Jusante').lower()=='sim' else False,
                            "forehead_damagemon": True if a.get('Testa ou ala danificada - Jusante') and a.get('Testa ou ala danificada - Jusante').lower()=='sim' else False,
                            "tube_damagemon": True if a.get('Tubulação danificada - Jusante') and a.get('Tubulação danificada - Jusante').lower()=='sim' else False,
                            "box_damagemon": True if a.get('Caixa danificada - Jusante') and a.get('Caixa danificada - Jusante').lower()=='sim' else False,
                            "erosionmon": True if a.get('Erosão - Jusante') and a.get('Erosão - Jusante').lower()=='sim' else False,
                            "brokenmon": True if a.get('Fissuras/trincas - Jusante') and a.get('Fissuras/trincas - Jusante').lower()=='sim' else False,
                            "cover_damagemon": True if a.get('Tampa Danificada/Inexist - Jusante') and a.get('Tampa Danificada/Inexist - Jusante').lower()=='sim' else False,                                            
                        })
            
#             rep_jusante.save()
        else:
            rep_jusante = rep_jusante.first()
    except Exception as e:
        print('Linha:',index+2,'Ficha:',rep,"Elemento Mon:",rep_montante,"Elemento Jus:",rep_jusante)
        print(e)
        a['linha_errada'] = index+2
#         erros.append(a)
        pass

  0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
Reporting.objects.filter(occurrence_type=occurrence_type_elemento,form_data__code=values[1].get('Código Montante'))

OperationalError: SSL connection has been closed unexpectedly
